In [1]:
from dateutil.parser import parse 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet as pr
#import mplcursors
from datetime import datetime
 
import warnings
warnings.filterwarnings("ignore")

Matplotlib created a temporary config/cache directory at C:\Users\MATHEU~1\AppData\Local\Temp\matplotlib-3fef_e4o because the default path (C:\Users\Matheus Alves\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
C:\Users\Matheus Alves\Documents\git_tcc\time_series_precipitation\ts\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


# Carregando Base de Dados

In [83]:
rainfall = pd.read_csv(r"C:\Users\Matheus Alves\Documents\git_tcc\time_series_precipitation\Dataset\dados_chuvas1.csv", parse_dates=True)

In [91]:
rainfall['Date'] = pd.to_datetime(rainfall['Date'], format="%Y/%m/d")
rainfall['Date'] = rainfall['Date'].dt.strftime('%Y-%m')

In [93]:
rainfall

,Date,Precipitation
0,1984-01,259.3
1,1984-02,32.5
2,1984-03,54.0
3,1984-04,96.8
4,1984-05,113.0
...,...,...
452,2021-09,39.5
453,2021-10,91.9
454,2021-11,98.6
455,2021-12,127.3


In [94]:
train, test = rainfall.iloc[:-14], rainfall.iloc[-15:]